<h1> <center> Create dataset 4 ML classification </center> </h1>

in this notebook you will find : 

* **Step 1** : feature selection > remove postoperative features  `TBC`
* **Step 2** : building target > based on two columns `IMMEDIATE_EXTUBATION` and `SECONDARY_EXTUBATION`
* **Step 3** : select a subset of patient > drop useless patients
* **Step 4** : export the new data ready for feature engineering > folder : `data/dim_patient_donor.csv`

In [35]:
import pandas as pd
import numpy as np 
dim_patient = pd.read_csv("../../../data/dim_patient.csv")

### Remove postoperative features

In [36]:
feat_fct_type = pd.read_csv('../features_config/features_functional_type.csv')
postoperative_features =feat_fct_type[feat_fct_type.type == 'postoperative'].features.tolist()


postoperative_features = [i.lower() for i in postoperative_features]
dim_patient.drop(postoperative_features, axis = 1, inplace = True)

### Build target

In [37]:
dim_patient["target"]  = np.nan
dim_patient["target"][(dim_patient["immediate_extubation"] == 1) & (dim_patient["secondary_intubation"] == 1)] = "unsuccessful IE"
dim_patient["target"][(dim_patient["immediate_extubation"] == 1) & (dim_patient["secondary_intubation"] == 0)] = "successful IE"

# -> IE : immediate extubation after lung transplantation 
#  if secondary intubation == 1: IE has been unsuccessful

/home/vagrant/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/vagrant/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Check 

In [38]:
dim_patient[dim_patient.target.notnull()][['target', 'secondary_intubation', 'immediate_extubation']].head()

,target,secondary_intubation,immediate_extubation
1,successful IE,0.0,1.0
5,successful IE,0.0,1.0
12,successful IE,0.0,1.0
13,successful IE,0.0,1.0
15,successful IE,0.0,1.0


In [39]:
dim_patient.drop(['secondary_intubation','immediate_extubation'], inplace = True, axis = 1 )

### select subset of patients

In [40]:
dim_patient = dim_patient[dim_patient.target.notnull()]

In [41]:
dim_patient.shape

(135, 55)

In [42]:
dim_patient.target.value_counts()/len(dim_patient) * 100

successful IE      88.148148
unsuccessful IE    11.851852
Name: target, dtype: float64

## Join donor features

In [43]:
dim_donor = pd.read_csv("../../../data/dim_donor.csv")
dim_patient_donor =  pd.merge(dim_patient, dim_donor, how = 'left', on = 'id_patient')
#export 
dim_patient_donor.to_csv("../../../data/dim_patient_donor.csv", index = False)